# Collect metadata from CrossRef

Intended to collect DOI metadata in batches from CrossRef via habanero, but not working atm

issue reported

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import datetime, time, sys
import json
import urllib.parse
import pandas as pd
import configparser
from pprint import pprint
from itertools import zip_longest
from tqdm import tqdm, tqdm_notebook

from ATB.ATB.DBConnection import DBConnection
from habanero import Crossref

tqdm.pandas(tqdm_notebook)

In [3]:
db_path = 'data/main.db'
db_table = 'pkp_crossref'
db_col_names = ['doi', 'timestamp', 'cr_object']
db_col_types = ['TEXT', 'TEXT', 'TEXT']
unique_col = "doi"

db_con = DBConnection(db_path=db_path,
                      db_table=db_table,
                      db_col_names=db_col_names,
                      db_col_types=db_col_types,
                      unique_col=unique_col)

In [4]:
cr = Crossref()

In [5]:
full_df = pd.read_csv("PKP_temp.csv", encoding = 'utf8')
dois = full_df[full_df.doi_resolve_status == "200"].doi

In [6]:
def parse_response(doi, now, cr_response):

    row = {'doi':str(doi),
           'timestamp':str(now),
           'cr_object':json.dumps(cr_response),
          }
    
    return row

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [47]:
a[0]

('10.20983/noesis.2015.1.2',
 '10.5195/jffp.2013.581',
 '10.7146/hjlcb.v7i12.25067',
 '10.7146/math.scand.a-12293',
 '10.12681/makedonika.1099',
 '10.5565/rev/dag.246',
 '10.5565/rev/athenead/v1n3.72',
 '10.5007/1984-8412.2017v14n1p1767',
 '10.21930/rcta.vol12_num1_art:214',
 '10.15150/lt.2003.1517',
 '10.21009/0305020312',
 '10.12681/makedonika.1071',
 '10.14714/CP12.1031',
 '10.1590/S0080-62342005000400007',
 '10.19030/ajhs.v3i2.6945',
 '10.21802/gmj.2017.2.6',
 '10.18592/tarbiyah.v5i2.985',
 '10.5489/cuaj.1787',
 '10.3402/ijch.v62i3.17559',
 '10.18002/ehf.v0i13.4515',
 '10.21930/rcta.vol18_num2_art:630',
 '10.13102/cl.v18i1.1657',
 '10.15844/pedneurbriefs-11-7-5',
 '10.15421/40260405',
 '10.11603/2415-8798.2016.4.7136',
 '10.5565/rev/papers.609',
 '10.5784/9-2-458',
 '10.26340/muallim.v6i24.1157',
 '10.5195/reviberoamer.2011.6791',
 '10.18764/2178-2865.v17n1p27-36',
 '10.3402/tellusa.v52i1.12254',
 '10.24109/2176-6673.emaberto.30i98.3240',
 '10.3402/ejpt.v4i0.20650',
 '10.21548/1-2-

In [72]:
try:
    res = cr.works(ids=a[0][0:4], cursor="*")
except:
    print('error')
    res = None

In [73]:
res

[{'message': {'DOI': '10.20983/noesis.2015.1.2',
   'ISSN': ['0188-9834', '2395-8669'],
   'URL': 'http://dx.doi.org/10.20983/noesis.2015.1.2',
   'author': [{'affiliation': [], 'family': 'Castro Lugo', 'given': 'David'},
    {'affiliation': [], 'family': 'Huesca Reynoso', 'given': 'Luis'},
    {'affiliation': [], 'family': 'Zamarrón Otzuca', 'given': 'Nathalia'}],
   'container-title': ['Nóesis. Revista de Ciencias Sociales y Humanidades'],
   'content-domain': {'crossmark-restriction': False, 'domain': []},
   'created': {'date-parts': [[2016, 3, 1]],
    'date-time': '2016-03-01T20:38:34Z',
    'timestamp': 1456864714000},
   'deposited': {'date-parts': [[2016, 9, 16]],
    'date-time': '2016-09-16T19:26:42Z',
    'timestamp': 1474054002000},
   'indexed': {'date-parts': [[2017, 10, 27]],
    'date-time': '2017-10-27T14:01:44Z',
    'timestamp': 1509112904464},
   'is-referenced-by-count': 0,
   'issn-type': [{'type': 'print', 'value': '0188-9834'},
    {'type': 'electronic', 'value

In [50]:
doi_groups = list(grouper(dois, 100))
for i, i_dois in tqdm(enumerate(doi_groups), total=len(doi_groups)):
    now = datetime.datetime.now()
    
    try:
        cr_response = cr.works(ids=i_dois)
    except:
        pass
    
    for response in cr_response:
        # Create DB entry
        row = parse_response(response['message']['DOI'], now, response['message'])
        db_con.save_row(row)
    
    # Do some nice API things
    if False:
        new = datetime.datetime.now()
        delta = new - now
        m, s = divmod(i_max-i, 60)
        h, m = divmod(m, 60)

        if delta.seconds < 1:
            time.sleep(1- delta.total_seconds())


  0%|          | 0/1216 [00:00<?, ?it/s]
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/asura/.virtualenvs/altmetrics/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/asura/.virtualenvs/altmetrics/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




HTTPError: 404 Client Error: Not Found for url: https://api.crossref.org/works/10.15150/lt.2003.1517